More about LangChain RAG: https://learn.deeplearning.ai/courses/langchain-chat-with-your-data/

## Dependencies

pip install transformers

pip install langchain, langchain-huggingface, langchain-community

pip install faiss-cpu

In [19]:
# Import dependencies
from langchain_huggingface import HuggingFacePipeline, ChatHuggingFace
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
    )
from langchain.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import AsyncHtmlLoader
from langchain_community.document_transformers import BeautifulSoupTransformer
from langchain.chains import RetrievalQA

## Load LLM and simple QA



In [20]:
# Load LLM
model_id = "unsloth/Llama-3.2-1B-Instruct"
# microsoft/Phi-4-mini-instruct
# google/gemma-3-1b-it
# meta-llama/Llama-3.2-1B-Instruct
# unsloth/Llama-3.2-1B-Instruct

# LLM
llm = HuggingFacePipeline.from_model_id(
    model_id=model_id,
    task="text-generation",
    pipeline_kwargs=dict(
        max_new_tokens=512,
        do_sample=False,
        repetition_penalty=1.03,
    ),
)


# Chat
chat = ChatHuggingFace(llm=llm, verbose=True)

Device set to use cuda:0


In [22]:
llm.invoke("What is SmartLab?")

C:\Users\chan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\transformers\generation\configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
C:\Users\chan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\transformers\generation\configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


"What is SmartLab? SmartLab is a company that specializes in developing and manufacturing high-quality, innovative laboratory equipment for the scientific community. Their products are designed to make scientific research more efficient, accurate, and cost-effective.\n\nSmartLab's products include a wide range of laboratory instruments, such as microscopes, spectrometers, and centrifuges, as well as software solutions for data analysis and visualization. Their equipment is known for its high precision, reliability, and ease of use, making it accessible to researchers from various disciplines, including biology, chemistry, physics, and more.\n\nSome of the key features of SmartLab's products include:\n\n* High-resolution imaging and spectroscopy\n* Advanced data analysis and visualization tools\n* Compact and portable designs\n* Easy-to-use interfaces and software\n* Compatibility with a wide range of operating systems\n\nOverall, SmartLab is a leading provider of laboratory equipment a

In [23]:
messages = [
    ("system", "You are a helpful assistant."),
    ("human", "What is SmartLab?"),
]

chat.invoke(messages)

C:\Users\chan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\transformers\generation\configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
C:\Users\chan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\transformers\generation\configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


AIMessage(content="<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 27 Mar 2025\n\nYou are a helpful assistant.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nWhat is SmartLab?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nSmartLab is a platform that allows users to create and manage their own virtual labs, where they can conduct experiments, simulations, and research in a safe and controlled environment. It's often used in educational settings, such as schools and universities, to teach various scientific concepts and skills.\n\nWith SmartLab, users can create virtual labs that mimic real-world experiments, allowing students to explore and learn about different scientific principles and phenomena in a hands-on way. The platform provides a range of features, including:\n\n1. Virtual lab environments: Users can create and customize their own virtual labs, including setting up equipment, adding materials, 

## Install Beautiful soup
pip install beautifulsoup4

https://pypi.org/project/beautifulsoup4/

# RAG from webpage

In [40]:
# Load documents from webpage
urls = ["https://www.metropolia.fi/fi/tutkimus-kehitys-ja-innovaatiot/yhteistyoalustat/smart-lab"]
loader_html = AsyncHtmlLoader(urls)
docs_html = loader_html.load()

bs_transformer = BeautifulSoupTransformer()
docs_transformed = bs_transformer.transform_documents(docs_html, tags_to_extract=["div"])

# Split it into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
docs_transformed_split = text_splitter.split_documents(docs_transformed)


print(len(docs_transformed_split))
print(len(docs_transformed[0].page_content))

Fetching pages: 100%|##########| 1/1 [00:00<00:00,  7.61it/s]

18
15864


### Another alternative to copy the content on another website

In [45]:
# Load documents from webpage
urls = ["https://www.deeplearning.ai/the-batch/agentic-design-patterns-part-2-reflection/"]
loader_html = AsyncHtmlLoader(urls)
docs_html = loader_html.load()

bs_transformer = BeautifulSoupTransformer()
docs_transformed = bs_transformer.transform_documents(docs_html, tags_to_extract=["div"])

# Split it into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
docs_transformed_split = text_splitter.split_documents(docs_transformed)


print(len(docs_transformed_split))
print(len(docs_transformed[0].page_content))

Fetching pages: 100%|##########| 1/1 [00:00<00:00,  3.04it/s]

6
5225


In [46]:
# create the open-source embedding function
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

# load it into FAISS
html_db = FAISS.from_documents(docs_transformed_split, embedding_function)

In [47]:
# Retriever

retriever_html = html_db.as_retriever()

qa_web = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever_html,    
    verbose=True
)

# Retrival QA

In [48]:
# Retrieval QA 
from time import time
def qa_rag(qa, query):
    print(f"Query: {query}\n")
    time_1 = time()
    result = qa.run(query)
    time_2 = time()
    print(f"Inference time: {round(time_2-time_1, 3)} sec.")
    print("\nResult:", result)

In [39]:
query = "Who are involved in the project SmartLab?"
qa_rag(qa_web, query)

Query: Who are involved in the project SmartLab?



> Entering new RetrievalQA chain...


C:\Users\chan\AppData\Local\Temp\ipykernel_16752\556439366.py:6: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa.run(query)
C:\Users\chan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\transformers\generation\configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
C:\Users\chan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\transformers\generation\configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or


> Finished chain.
Inference time: 1.088 sec.

Result: Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

(/sites/default/files/2021-12/sl_049_nettikoko.jpg) SmartLab-keittiö   (/sites/default/files/2021-12/sl_108_nettikoko.jpg)                (/sites/default/files/2021-12/sl_108_nettikoko.jpg) SmartLab-paneeli   (/sites/default/files/2021-12/sl_015_nettikoko.jpg)                (/sites/default/files/2021-12/sl_015_nettikoko.jpg) SmartLab sähkökaappi   (/sites/default/files/2021-12/sl_059_nettikoko.jpg)                (/sites/default/files/2021-12/sl_059_nettikoko.jpg) Valokatkaisimia SmartLabissa   (/sites/default/files/2021-12/sl_060_nettikoko.jpg)                (/sites/default/files/2021-12/sl_060_nettikoko.jpg) SmartLab valaistus Ota yhteyttä Teknologiapäällikkö Harri Hahkala p. 040 082 8636 harri.hahkala [at] metropolia.fi (harri[dot]hahkala[at]metropolia[dot]fi) Innov

In [49]:
query = "What is agentic reasoning?"
qa_rag(qa_web, query)

Query: What is agentic reasoning?



> Entering new RetrievalQA chain...


C:\Users\chan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\transformers\generation\configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
C:\Users\chan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\transformers\generation\configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(



> Finished chain.
Inference time: 1.991 sec.

Result: Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Read "Agentic Design Patterns Part 3, Tool Use" (https://www.deeplearning.ai/the-batch/agentic-design-patterns-part-3-tool-use/?ref=dl-staging-website.ghost.io) Read "Agentic Design Patterns Part 4: Planning" (https://www.deeplearning.ai/the-batch/agentic-design-patterns-part-4-planning/?ref=dl-staging-website.ghost.io) Read "Agentic Design Patterns Part 5: Multi-Agent Collaboration" (https://www.deeplearning.ai/the-batch/agentic-design-patterns-part-5-multi-agent-collaboration/?ref=dl-staging-website.ghost.io) Share Subscribe to The Batch Stay updated with weekly AI News and Insights delivered to your inbox Courses The Batch Community Careers About

✨ New course! Enroll in   Vibe Coding 101 with Replit (https://bit.ly/4l8QyZ5) Explore Courses AI Newsletter The Batch And

In [50]:
# Clear RAM cache
#del pipe

# Clear CUDA cache
import torch
torch.cuda.empty_cache()